# Meaningful Play Score Assigner

This program is designed to take an adjacency matrix of a topology of non-looping, non-backtracking linear choices, and apply q-learning to determine how meaningful the set of choices would be from the perspective of the actor.

This is the "upside down" variant, that starts at the endings and works its way up to the start.

## Setup

Initial set up before we begin:

#### Import Statements

Here all libraries that we use will be imported

In [120]:
from math import *
from decimal import Decimal
import numpy as np
import random
import xlsxwriter
import xml.dom.minidom as minidom

#### Gather User Input

We'll need to know the input file for the graph and the number of layers

TODO: (Can modify layer and ending counts to be automatically calculated from adjacency matrix)

In [121]:
filename = "NoIntegrated.xml"#"SteinsGateChoicesAdjusted.xml" #input("Please input the name of the topology .xml file you want to score: ")
layers = 5#59 #int(input("Please input the number of non-ending layers your topology has: "))
endings = 4#6 #int(input("Please input the number of endings in your topology: "))

#### Create State Mapping and Adjacency matrix

The states need to be put into a map for identification purposes, and an adjacency matrix can also be generated from the same file.

In [122]:
#info: https://www.guru99.com/manipulating-xml-with-python.html#3

#Mapping for the states
location_to_state = {}

#Map to help finish populating (imperfect copy of location_to_state)
tempMap = {}

doc = minidom.parse(filename)

#load in all the relevant elements to a list called "cells"
cells = doc.getElementsByTagName("mxCell")

#list to hold nodes
nodes = []
#list to hold connections between nodes
connections = []
#Number to keep track of how many endings have been added
endingIndex = 1

#loop through cells to sort ellipses and endArrows into their respective lists
for label in cells:
    if(label.getAttribute("style") != ""):
        if(label.getAttribute("style").startswith("ellipse")):
            nodes.append(label)
        elif(label.getAttribute("style").startswith("endArrow")):
            connections.append(label)

#declare empty adjacency matrix to populate with data on the graph
rewards = []
rewards_forward = []

#populate matrix and map simulatneously as you iterate through the nodes
for i in range(len(nodes)):
    rewards.append([])
    rewards_forward.append([])
    if "ending" in nodes[i].getAttribute("value"):
        location_to_state["E" + str(endingIndex)] = i
        print("E" + str(endingIndex) + " is: " + nodes[i].getAttribute("value"))
        endingIndex = endingIndex + 1
    elif "start" in nodes[i].getAttribute("value"):
        location_to_state["Start"] = i
    else:
        location_to_state[nodes[i].getAttribute("id")] = i
    
    tempMap[nodes[i].getAttribute("id")] = i
    
    for j in range(len(nodes)):
        rewards[i].append(0)
        rewards_forward[i].append(0)

#finish populating the adjacency matrix with data on the connections between nodes
for link in connections:
    x = tempMap[link.getAttribute("target")]
    y = tempMap[link.getAttribute("source")]
    rewards[x][y] = 1
    
#FOR Upside-down, make copy with normal directions for weighting algorithm to use:
for link in connections:
    x = tempMap[link.getAttribute("source")]
    y = tempMap[link.getAttribute("target")]
    rewards_forward[x][y] = 1

#convert matrix into nparray and print it and the map for manual inspection
rewards = np.asarray(rewards)
print("Matrix:\n")
print(rewards)
print("\nMap:\n")
print(location_to_state)

print(tempMap)

# Map indices to locations
state_to_location = dict((state,location) for location,state in location_to_state.items())

E1 is: ending_1
E2 is: ending_2
E3 is: ending_3
E4 is: ending_4
Matrix:

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 0 0 0

#### Define q-learning Functions

In [123]:
class QAgent():
    
    def __init__(self, alpha, gamma, epsilon, location_to_state, rewards, state_to_location, Q):
        """ Initialize alpha, gamma, epsilon, states, actions, rewards, and Q-values
        """
        self.gamma = gamma  
        self.alpha = alpha 
        self.epsilon = epsilon
        
        self.location_to_state = location_to_state
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        self.Q = Q
        
    def training(self, start_location, end_location, iterations):
        """Training the system in the given environment to move from a start state to an end state
        """
        rewards_new = np.copy(self.rewards)
        
        #set reward for end state to 100 to incentivize reaching desired end
        ending_state = self.location_to_state[end_location]
        rewards_new[ending_state, ending_state] = 100

        #Loop for specified number of iterations
        for i in range(iterations):
            
            #initialize current state as the start state
            #current_state = self.location_to_state[start_location]
            
            #Randomly pick a state to observe
            current_state = np.random.randint(0,len(self.rewards)) 
            
            #counter to make sure it hits a dead end after a while
            counter = 0
            
            #infinite loop runs through route, updating until it hits a dead-end
            while(True):
                
                #Iterate counter. If it hits the limit, break the loop and print error message
                counter += 1
                if(counter == limit):
                    print("Error: Hit limit before reaching an end while training")
                    break
                
                #Construct list of possible actions
                playable_actions = []
                
                for j in range(len(self.rewards)):
                    if rewards_new[current_state,j] > 0:
                        playable_actions.append(j)

                #Only run updates if observed state has performable actions
                if playable_actions:
                    #Decide whether to random walk or follow standard policy
                    if random.uniform(0, 1) < epsilon:
                        next_state = np.random.choice(playable_actions)
                    else:
                        next_state = np.argmax(rewards_new[current_state,])

                    #Calculate temporal difference
                    TD = rewards_new[current_state,next_state] + \
                            self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state,next_state]

                    #updates Q-value using Bellman equation
                    self.Q[current_state,next_state] += self.alpha * TD
                    
                    #check if agent is at desired ending
                    if next_state == current_state:
                        break
                    
                    #update current state to move forward
                    current_state = next_state
                else:
                    break

        route = [start_location]
        next_location = start_location
        
        # Get the route 
        return self.get_optimal_route(start_location, end_location, next_location, route, self.Q)
        
    # Get the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        
        #set counter to break if it learns wrong route
        counter = 0
        
        while(next_location != end_location):
            #Iterate counter. If it hits the limit, break the loop and print error message
            counter += 1
            if(counter >= limit):
                print("Error: Failed to learn correct route")
                return None
            
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location            

        return route
    
#Take set of q-tables and average them into one q-table
def qaverage(table_set):
    num = 0
    output_table = table_set[0].copy()
    for i in range(len(table_set[0][0])):
        for j in range(len(table_set[0])):
            for k in range(len(table_set)):
                num += table_set[k][j][i]
            output_table[j][i] = num / len(table_set)
            num = 0

    return output_table
    
# Initialize parameters
gamma = 0.75 # Discount factor (discounts previous rewards)
alpha = 0.9 # Learning rate
epsilon = 0.2 # Exploration vs exploitation percentage

limit = 100000 # number of steps until a dead end is hit

## q-learning

So now that all the setup has been done, we can start our q-learning algorithm, then move on to processing its output.

#### Define q-learning Execution Functions

We need a couple of functions to handle our q-learning, since we need to execute multiple times

In [124]:
#Handle all q-learning for a given topology
def qmaster(start_state, output_tables):
    #array to store the final Q-Table of each 1000 iterations
    qtables = []
    for i in range(100):
        qagent = QAgent(alpha, gamma, epsilon, location_to_state, rewards,  state_to_location, 
                        np.array(np.zeros([len(location_to_state),len(location_to_state)])))
    
        training_results = qagent.training(start_state, "Start", 1000)
        
        #TODO: REMOVE
        print("Done with cycle", i, " out of 100")
        if training_results is not None:
            qtables.append(qagent.Q)

    if len(qtables) > 0:
        output_tables.append(qaverage(qtables))

#### Run q-learning algorithm

Finally, we can run q-master and store the output

In [125]:
#an array to hold the outputs of q-master.
averaged_tables = []

#run qmaster for each ending state
for i in range(endings):
    qmaster("E" + str(i + 1), averaged_tables)

Done with cycle 0  out of 100
Done with cycle 1  out of 100
Done with cycle 2  out of 100
Done with cycle 3  out of 100
Done with cycle 4  out of 100
Done with cycle 5  out of 100
Done with cycle 6  out of 100
Done with cycle 7  out of 100
Done with cycle 8  out of 100
Done with cycle 9  out of 100
Done with cycle 10  out of 100
Done with cycle 11  out of 100
Done with cycle 12  out of 100
Done with cycle 13  out of 100
Done with cycle 14  out of 100
Done with cycle 15  out of 100
Done with cycle 16  out of 100
Done with cycle 17  out of 100
Done with cycle 18  out of 100
Done with cycle 19  out of 100
Done with cycle 20  out of 100
Done with cycle 21  out of 100
Done with cycle 22  out of 100
Done with cycle 23  out of 100
Done with cycle 24  out of 100
Done with cycle 25  out of 100
Done with cycle 26  out of 100
Done with cycle 27  out of 100
Done with cycle 28  out of 100
Done with cycle 29  out of 100
Done with cycle 30  out of 100
Done with cycle 31  out of 100
Done with cycle 32

Done with cycle 66  out of 100
Done with cycle 67  out of 100
Done with cycle 68  out of 100
Done with cycle 69  out of 100
Done with cycle 70  out of 100
Done with cycle 71  out of 100
Done with cycle 72  out of 100
Done with cycle 73  out of 100
Done with cycle 74  out of 100
Done with cycle 75  out of 100
Done with cycle 76  out of 100
Done with cycle 77  out of 100
Done with cycle 78  out of 100
Done with cycle 79  out of 100
Done with cycle 80  out of 100
Done with cycle 81  out of 100
Done with cycle 82  out of 100
Done with cycle 83  out of 100
Done with cycle 84  out of 100
Done with cycle 85  out of 100
Done with cycle 86  out of 100
Done with cycle 87  out of 100
Done with cycle 88  out of 100
Done with cycle 89  out of 100
Done with cycle 90  out of 100
Done with cycle 91  out of 100
Done with cycle 92  out of 100
Done with cycle 93  out of 100
Done with cycle 94  out of 100
Done with cycle 95  out of 100
Done with cycle 96  out of 100
Done with cycle 97  out of 100
Done wit

#### Get weights

Calculate and apply weights to each of the averaged q-tables

In [126]:
#Calculate weights
def weight_calculator(layers):
    #get slope
    slope = 1 / (layers * layers)
    #get sum
    sum = 0
    for i in range(layers):
        sum += (i * slope)
    
    #get amount to add to equal 1
    toAdd = (1 - sum) / layers
    
    #Finally, set up and return array of weights
    weights = []
    for i in range(layers):
        weights.append((i * slope) + toAdd)
        
    #print(weights)
    return weights

#list of visited nodes for weighting function to ignore
visited = []

#Apply weighting function to give high score to early states
def apply_weights_helper(directions, array, layers, endings):
    #zero out the start state subarray to eliminate any rewards placed there.
    for i in range(len(array)):
        directions[i][i] = 0
        array[location_to_state['Start']][i] = 0
    
    weights = weight_calculator(int(layers))
    #weights.reverse()
    apply_weights(directions, array, weights, location_to_state['Start'], 0)
    
def apply_weights(directions, array, weights, x, level):
    for i in range(len(array[x])):
        if(level < len(weights)):
            array[x][i] = array[x][i] * weights[level]
        if(directions[x][i] > 0 and i not in visited):
            print("Going to:" + str(x) + "," + str(i))
            visited.append(i)
            apply_weights(directions, array, weights, i, (level + 1))
            
for i in range(len(averaged_tables)):
    apply_weights_helper(rewards_forward, averaged_tables[i], layers, endings)
    print("Done with table ", i)
    
print(averaged_tables)

Going to:0,9
Going to:9,1
Going to:1,17
Going to:17,13
Going to:13,2
Going to:13,4
Going to:13,6
Going to:13,8
Going to:17,14
Going to:17,15
Going to:17,16
Going to:1,18
Going to:1,19
Going to:1,20
Going to:9,3
Going to:9,5
Going to:9,7
Going to:0,10
Going to:0,11
Going to:0,12
Done with table  0
Done with table  1
Done with table  2
Done with table  3
[array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , 45.35      ,
        45.35      , 45.35      , 45.35      ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.     

#### Normalizing

Before we take the pairwise minkowski distance to get our score, we want to normalize our weighted q-tables.
We can do this by running each subarray of each table through the Softmax function.

In [127]:
#Softmax implementation modified from
#https://intellipaat.com/community/942/how-to-implement-the-softmax-function-in-python

def softmax(x): 
    """Compute softmax values for each sets of scores in x.""" 
    e_x = np.exp(x - np.max(x)) 
    return (e_x / e_x.sum(axis=0)).tolist()

def normalize(array, endings):
    processedList = []
    indices = []
    pair = []
    for i in range(len(array)):
        for j in range(len(array)):
            if(array[i][j] > 0):
                pair.append(i)
                pair.append(j)
                indices.append(pair.copy())
                pair = []
                processedList.append(array[i][j])
        
    processedList = softmax(processedList)
    for j in range(len(indices)):
        array[indices[j][0]][indices[j][1]] = processedList[j]
        
            
#generates excel spreadsheet containing all q-tables in a given path
def to_excel(qtables):
    """store data in excel
    """
    workbook = xlsxwriter.Workbook(filename + ".xlsx")

    #write each q-table to another worksheet
    for i in range(len(qtables)):
        worksheet = workbook.add_worksheet()
        for j in range(len(qtables[i])):
            for k in range(len(qtables[i])):
                worksheet.write(j, k, qtables[i][j][k])

    workbook.close()
            
for i in range(len(averaged_tables)):
    #normalize(averaged_tables[i], endings)
    to_excel(averaged_tables)

#### Calculate Minkowski Distances and Output the Score

We're finally ready to calculate the minkowski distance and output a meaningfulness score

In [128]:
#Minkowski Distance implementation altered from
#https://www.geeksforgeeks.org/minkowski-distance-python/

#convert array into 1D vector for ease of manipulation
def vectorize(input_array):
    output_array = []
    for i in range(len(input_array)):
        for j in range(len(input_array[0])):
            output_array.append(input_array[i][j])
            
    return output_array

#Calculate Minkowski distance between arrays
  
# Function distance between two points  
# and calculate distance value to given 
# root value(p is root value) 
def p_root(value, root): 
      
    root_value = 1 / float(root) 
    return round (Decimal(value) **
             Decimal(root_value), 3) 
  
def minkowski_distance(x, y, p_value): 
    # pass the p_root function to calculate 
    # all the values of vector in parallel 
    return (p_root(sum(pow(abs(a-b), p_value) 
            for a, b in zip(x, y)), p_value))

#vectorize averaged tables
vectors = []
for i in range(len(averaged_tables)):
    vectors.append(vectorize(averaged_tables[i]))
    
#calculate minkowski distances in a pairwise fashion
distances = []
acc = 0
for i in range(len(averaged_tables) - 1):
    for j in range(i + 1, len(vectors)):
        distance = minkowski_distance(vectors[i], vectors[j], 1) / 2
        distances.append(distance)
        acc += distance

print(acc / len(distances))

1872.506416666666666666666667
